In [ ]:
import csv
import re
import glob
import pandas as pd

path_str = "/project/work01/BTSJ_hr/5_Friend/*.xlsx"
paths = glob.glob(path_str)

In [ ]:
field_name = ['category','id1','utterance1','id2','utterance2','agreement','laugh']

category = 5

data =[]

In [ ]:
def cleansing(text):
    pattern = re.compile(r',,|‘.*?’|《.*?》|=|…|『|』|\“|\”|【|】|「.*?」|\[.*?\]|\?|<.*?>|〈.*?〉|\(.*?\)|#|{|}|!|！')
    text = re.sub(r'<([^>]*)>{>}', r'{\1}', text)
    text = re.sub(r'<([^>]*)>{<}', r'{\1}', text)
    text = pattern.sub('', text)
    text = re.sub(r'N{2}','', text)
    text = re.sub(r'，', r'、', text)
    text = re.sub(r'、+', r'、', text)
    text = re.sub(r'。+', r'。', text)
    text = re.sub(r'、。', r'。', text)
    text = re.sub(r'^[!"#$%&\'()*+-.,\/:;<=>?@[\]^_`{|}~、。ー 　]+','', text)
    return text

In [ ]:
with open('/project/work01/BTSJ_data/hr_5.csv','w',encoding='utf-8') as csvfile:
    for file_path in paths:
        current_data = {'category':category, 'id1':None, 'utterance1':None, 'id2':None, 'utterance2':None, 'agreement':None, 'laugh':None}
        df = pd.read_excel(file_path)
        df = df.drop(df.index[[0]])
        df = df.drop(df.columns[[0, 1, 2]], axis=1)
        df = df.drop(df.columns[5:], axis=1)
        last_line, last_row = df.shape
        print(last_line)
        for i in range(1, last_line):
            if i == 1:
                current_data['id2'] = df.iloc[i, 3]
                current_data['utterance2'] = cleansing(df.iloc[i,4])
            elif i == last_line-1:
                if current_data['id2'] == df.iloc[i, 3]:
                    current_data['utterance2'] += cleansing(df.iloc[i,4])
                    data.append(current_data.copy())
                else:
                    if current_data['id1'] is not None and current_data['id2'] is not None and current_data['id1'] != current_data['id2']:
                        data.append(current_data.copy())
                    current_data['id1'] = current_data['id2']
                    current_data['utterance1'] = current_data['utterance2']
                    current_data['id2'] = df.iloc[i, 3]
                    current_data['utterance2'] = cleansing(df.iloc[i,4])
                    data.append(current_data.copy())
            else:
                if current_data['id2'] == df.iloc[i, 3]:
                    current_data['utterance2'] += cleansing(df.iloc[i,4])
                else:
                    if current_data['id1'] is not None and current_data['id2'] is not None and current_data['id1'] != current_data['id2']:
                        data.append(current_data.copy())
                    current_data['id1'] = current_data['id2']
                    current_data['utterance1'] = current_data['utterance2']
                    current_data['id2'] = df.iloc[i, 3]
                    current_data['utterance2'] = cleansing(df.iloc[i,4])
    
    writer = csv.DictWriter(csvfile, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(data)
